<a href="https://colab.research.google.com/github/Jasmeetk05/2nd-year-project/blob/main/Copy_of_Untitled55.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------------------
# Step 1: Install dependencies
# ------------------------------
!pip install transformers datasets gradio evaluate -q

# ------------------------------
# Step 2: Import libraries
# ------------------------------
import gradio as gr
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ------------------------------
# Step 3: Load model
# ------------------------------
MODEL = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=1)
model.eval()

# ------------------------------
# Step 4: Define users (multi-user login)
# ------------------------------
USERS = {
    "student": "1234",
    "teacher": "abcd",
    "admin": "admin123"
}

# ------------------------------
# Step 5: Essay grading function
# ------------------------------
def grade_essay(question, answer):
    text = f"Question: {question} Answer: {answer}"
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)

    with torch.no_grad():
        outputs = model(**inputs)
        score = outputs.logits.squeeze().item()

    score = np.clip((score + 3) * 15, 0, 100)

    feedback = []
    if len(answer.split()) < 30:
        feedback.append("Your answer is too short. Try adding more details.")
    if question.lower().split()[0] not in answer.lower():
        feedback.append("Make sure your answer is closely related to the question.")
    if score > 70:
        feedback.append("Great job! Your answer is well-structured and detailed.")
    elif score > 40:
        feedback.append("Decent attempt, but you can improve by explaining more clearly.")
    else:
        feedback.append("Needs significant improvement. Try to include key concepts.")

    return f"Predicted Score: {score:.2f}/100\n\nFeedback: {' '.join(feedback)}"

# ------------------------------
# Step 6: Login & Logout Functions
# ------------------------------
def login(username, password, logged_in):
    username = username.strip().lower()
    password = password.strip()

    if username in USERS and USERS[username] == password:
        return gr.update(visible=False), gr.update(visible=True), f"✅ Welcome {username.title()}!", True
    else:
        return gr.update(visible=True), gr.update(visible=False), "❌ Invalid username or password", False

def logout(logged_in):
    return gr.update(visible=True), gr.update(visible=False), "👋 You have logged out.", False

# ------------------------------
# Step 7: Build Gradio App
# ------------------------------
with gr.Blocks(theme=gr.themes.Soft(primary_hue="pink", secondary_hue="cyan")) as demo:
    logged_in = gr.State(False)

    gr.Markdown("""
    <h1 style="color:#ff1493;">✨ Automatic Essay/Answer Grader ✨</h1>
    <p style="color:#00bfff;">Login to access the grading system.</p>
    """)

    # Login Screen
    with gr.Group(visible=True) as login_screen:
        username = gr.Textbox(label="👤 Username")
        password = gr.Textbox(label="🔑 Password", type="password")
        login_btn = gr.Button("Login 🚪")
        login_message = gr.Markdown("")

    # Main App (hidden until login)
    with gr.Group(visible=False) as app_screen:
        with gr.Row():
            question = gr.Textbox(label="📘 Enter Question", placeholder="e.g., Explain Newton's First Law of Motion")
        with gr.Row():
            answer = gr.Textbox(label="✍️ Enter Your Answer", placeholder="Type your answer here...", lines=6)

        submit_btn = gr.Button("🚀 Grade My Answer")
        output = gr.Textbox(label="📊 Results", lines=6)

        logout_btn = gr.Button("Logout 🔒")
        status_msg = gr.Markdown("")

        submit_btn.click(fn=grade_essay, inputs=[question, answer], outputs=output)

    # Button Actions
    login_btn.click(fn=login, inputs=[username, password, logged_in],
                    outputs=[login_screen, app_screen, login_message, logged_in])

    logout_btn.click(fn=logout, inputs=[logged_in],
                     outputs=[login_screen, app_screen, status_msg, logged_in])

# ------------------------------
# Step 8: Launch app
# ------------------------------
demo.launch(share=True)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6705de29fe2860fc8e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
